<a href="https://colab.research.google.com/github/abhinandan6123/AI-Projects/blob/main/DatapreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBRARIES

In [ ]:
import numpy as np
import IPython.display as display
from matplotlib import pyplot as plt
import pandas as pd

IMPORTING DATASET

In [ ]:
dataset = pd.read_csv('delhivery.csv')
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(x)

[['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 14.0 11.0
  11.9653]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 10.0 9.0
  9.759]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 16.0 7.0
  10.8152]
 ...
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 20.0 34.0
  20.7053]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 17.0 27.0
  18.8885]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 268.0 9.0
  8.8088]]


In [ ]:
print(y)

[ 1.27272727  1.11111111  2.28571429 ...  0.58823529  0.62962963
 29.77777778]


HANDLING MISSING DATA

In [11]:

numeric_columns = dataset.select_dtypes(include=[np.number]).columns
non_numeric_columns = dataset.select_dtypes(exclude=[np.number]).columns

In [14]:

numeric_imputer = SimpleImputer(strategy='mean')
dataset[numeric_columns] = numeric_imputer.fit_transform(dataset[numeric_columns])


In [15]:
# Forward fill for non-numeric columns (suitable for timestamps and categories)
dataset[non_numeric_columns] = dataset[non_numeric_columns].fillna(method='ffill')


<ipython-input-15-3e859ce3049b>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset[non_numeric_columns] = dataset[non_numeric_columns].fillna(method='ffill')


In [16]:
print(x)

[['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 14.0 11.0
  11.9653]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 10.0 9.0
  9.759]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 16.0 7.0
  10.8152]
 ...
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 20.0 34.0
  20.7053]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 17.0 27.0
  18.8885]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 268.0 9.0
  8.8088]]


In [18]:
date_columns = ['trip_creation_time', 'od_start_time', 'od_end_time', 'cutoff_timestamp']  # list your datetime columns
for col in date_columns:
    dataset[col] = pd.to_datetime(dataset[col], errors='coerce')


In [19]:
for col in date_columns:
    dataset[f'{col}_year'] = dataset[col].dt.year
    dataset[f'{col}_month'] = dataset[col].dt.month
    dataset[f'{col}_day'] = dataset[col].dt.day
    dataset[f'{col}_hour'] = dataset[col].dt.hour
    dataset[f'{col}_minute'] = dataset[col].dt.minute


In [20]:
# Example with one-hot encoding
dataset = pd.get_dummies(dataset, columns=['route_type', 'source_center', 'destination_center'], drop_first=True)


In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_columns = ['actual_distance_to_destination', 'actual_time', 'osrm_time', 'osrm_distance']  # add relevant columns
dataset[scaled_columns] = scaler.fit_transform(dataset[scaled_columns])


In [23]:
# Example using IQR for 'actual_distance_to_destination'
Q1 = dataset['actual_distance_to_destination'].quantile(0.25)
Q3 = dataset['actual_distance_to_destination'].quantile(0.75)
IQR = Q3 - Q1
dataset = dataset[(dataset['actual_distance_to_destination'] >= (Q1 - 1.5 * IQR)) &
        (dataset['actual_distance_to_destination'] <= (Q3 + 1.5 * IQR))]


In [26]:
from sklearn.model_selection import train_test_split

X = dataset  # replace 'target_column' with your target variable
y = dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
dataset.to_csv('preprocessed_data.csv', index=False)


In [32]:
from google.colab import files


In [33]:
print(dataset.head())

       data         trip_creation_time  \
0  training 2018-09-20 02:35:36.476840   
1  training 2018-09-20 02:35:36.476840   
2  training 2018-09-20 02:35:36.476840   
3  training 2018-09-20 02:35:36.476840   
4  training 2018-09-20 02:35:36.476840   

                                 route_schedule_uuid                trip_uuid  \
0  thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...  trip-153741093647649320   
1  thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...  trip-153741093647649320   
2  thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...  trip-153741093647649320   
3  thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...  trip-153741093647649320   
4  thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...  trip-153741093647649320   

                  source_name               destination_name  \
0  Anand_VUNagar_DC (Gujarat)  Khambhat_MotvdDPP_D (Gujarat)   
1  Anand_VUNagar_DC (Gujarat)  Khambhat_MotvdDPP_D (Gujarat)   
2  Anand_VUNagar_DC (Gujarat)  Khambhat_MotvdDPP_D (Gujarat)   
3  A

In [35]:
print(x)

[['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 14.0 11.0
  11.9653]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 10.0 9.0
  9.759]
 ['training' '2018-09-20 02:35:36.476840'
  'thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3297ef' ... 16.0 7.0
  10.8152]
 ...
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 20.0 34.0
  20.7053]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 17.0 27.0
  18.8885]
 ['training' '2018-09-20 16:24:28.436231'
  'thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d55f058' ... 268.0 9.0
  8.8088]]


In [36]:
print(dataset.tail())

            data         trip_creation_time  \
144862  training 2018-09-20 16:24:28.436231   
144863  training 2018-09-20 16:24:28.436231   
144864  training 2018-09-20 16:24:28.436231   
144865  training 2018-09-20 16:24:28.436231   
144866  training 2018-09-20 16:24:28.436231   

                                      route_schedule_uuid  \
144862  thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d5...   
144863  thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d5...   
144864  thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d5...   
144865  thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d5...   
144866  thanos::sroute:f0569d2f-4e20-4c31-8542-67b86d5...   

                      trip_uuid                 source_name  \
144862  trip-153746066843555182  Sonipat_Kundli_H (Haryana)   
144863  trip-153746066843555182  Sonipat_Kundli_H (Haryana)   
144864  trip-153746066843555182  Sonipat_Kundli_H (Haryana)   
144865  trip-153746066843555182  Sonipat_Kundli_H (Haryana)   
144866  trip-153746066843555182  So